In [56]:
import ipywidgets as widgets
from IPython.display import clear_output
from intermine.webservice import Service
from IPython.core.display import display, HTML
import tabulate
import json
import re
from ipywidgets import Button, GridBox, Layout, ButtonStyle
import csv
import pandas as pd

## RUN ME FIRST
import primer3.bindings as bindings

primer__ = re.compile("PRIMER_((LEFT)|(RIGHT))_[0-5]", re.I)
entered_value = "SMESG000027291.1"

def designPrimers(sequence):
    PCR_primers = bindings.designPrimers(
    {
        'SEQUENCE_ID': 'MH1000',
        'SEQUENCE_TEMPLATE': sequence,
        'SEQUENCE_INCLUDED_REGION': [10,len(sequence)-10]
    },
    {
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_INTERNAL_OLIGO': 1,
        'PRIMER_INTERNAL_MAX_SELF_END': 8,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 20.0,
        'PRIMER_MAX_GC': 80.0,
        'PRIMER_MAX_POLY_X': 100,
        'PRIMER_INTERNAL_MAX_POLY_X': 100,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 12,
        'PRIMER_MAX_SELF_END': 8,
        'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        'PRIMER_PAIR_MAX_COMPL_END': 8,
        'PRIMER_PRODUCT_SIZE_RANGE': [[750,1000],],
    })

    qPCR_primers = bindings.designPrimers(
    {
        'SEQUENCE_ID': 'MH1000',
        'SEQUENCE_TEMPLATE': sequence,
        'SEQUENCE_INCLUDED_REGION': [10,len(sequence)-10]
    },
    {
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_INTERNAL_OLIGO': 1,
        'PRIMER_INTERNAL_MAX_SELF_END': 8,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 20.0,
        'PRIMER_MAX_GC': 80.0,
        'PRIMER_MAX_POLY_X': 100,
        'PRIMER_INTERNAL_MAX_POLY_X': 100,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 12,
        'PRIMER_MAX_SELF_END': 8,
        'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        'PRIMER_PAIR_MAX_COMPL_END': 8,
        'PRIMER_PRODUCT_SIZE_RANGE': [[100,200],],
    })
    
    
    return PCR_primers, qPCR_primers


def gene_output(search_query):
    service = Service("http://planmine.mpi-cbg.de/planmine/service")

    query = service.new_query("Gene")
    query.add_view(
    "transcripts.sequence.residues", "transcripts.primaryIdentifier",
    "primaryIdentifier", "symbol")

    query.add_constraint("Gene", "LOOKUP", search_query, "", code = "A")

    return query

    

def handle_primer_output(input_primer_dict):
    primer_pairs = dict()
    for x in input_primer_dict:
        if primer__.search(x):
            primer_pairs[x.split("_")[2]] = {}   
    
    for x in primer_pairs:
        for LoR in ["LEFT", "RIGHT"]: 
            penalty = input_primer_dict.get(f"PRIMER_{LoR}_{x}_PENALTY")
            sequence = input_primer_dict.get(f"PRIMER_{LoR}_{x}_SEQUENCE")
            boundaries = input_primer_dict.get(f"PRIMER_{LoR}_{x}")
            melting_temp = input_primer_dict.get(f"PRIMER_{LoR}_{x}_TM")
            GC_content = input_primer_dict.get(f"PRIMER_{LoR}_{x}_GC_PERCENT")
            product_size = input_primer_dict.get(f'PRIMER_PAIR_{x}_PRODUCT_SIZE')
            primer_size = input_primer_dict.get(f"PRIMER_{LoR}_{x}_GC_PERCENT")
        
            primer_pairs[x].update({LoR : {"penalty" : penalty, 
                                     "sequence" : sequence,
                                     "boundaries" : boundaries,
                                     "TM" : melting_temp, 
                                     "GC" : GC_content,
                                          "size": boundaries[1],
                                          "product_size" : product_size} })
    return primer_pairs
        
def calcPCRout(sender):
    global selected_transcript
    sequence = all_entries.get(selected_transcript)
    PCR_primers, qPCR_primers = designPrimers(sequence)
    
    clear_output()
    
    display(HTML(f'<h1>{selected_transcript}</h1>'))
    display(HTML(f'<code>{sequence}</code>'))
    
    
    PCR_primers_dict = handle_primer_output(PCR_primers)
    qPCR_primers_dict = handle_primer_output(qPCR_primers)
    
    primer_dicts = {"qPCR_primers" : qPCR_primers_dict, 
                    "PCR_primers" : PCR_primers_dict}
    
   
    
    qPCR_out = widgets.Output()
    PCR_out = widgets.Output()
    

    with qPCR_out:
        for x, items in qPCR_primers_dict.items():
            display(HTML(f"<h3>Primer pair #{int(x)+1}"))
            
            tables = list()
            
            for LoR in ["LEFT", "RIGHT"]:
                boundaries = items.get(LoR).get("boundaries")
                if LoR == "LEFT":
                    fr = "Forward"
                elif LoR == "RIGHT":
                    fr = "Reverse"
                table = [[fr, "PRIMER"],
                         ["Sequence", items.get(LoR).get("sequence")],
                     ["From... To", f"{boundaries[0]}... {boundaries[0]+boundaries[1]}"],
                     ["TM", items.get(LoR).get("TM")], 
                     ["GC", items.get(LoR).get("GC")],
                     ["Amplicon Length", items.get(LoR).get("product_size")],
                         ["Primer Size", items.get(LoR).get("size")],
                    ]
                inter_out = widgets.Output()
                with inter_out:
                    display(HTML(tabulate.tabulate(table, tablefmt='html')))

                tables.append(inter_out)
        
            griddy_2 = widgets.GridBox(children=tables,
            layout=Layout(
                width='100%',
                grid_template_rows='auto',
                grid_template_columns='50% 50%',))
            display(griddy_2)
            
    with PCR_out: 
        for x, items in PCR_primers_dict.items():
            display(HTML(f"<h3>Primer pair #{int(x)+1}"))
            
            tables = list()
            
            for LoR in ["LEFT", "RIGHT"]:
                boundaries = items.get(LoR).get("boundaries")
                if LoR == "LEFT":
                    fr = "Forward"
                elif LoR == "RIGHT":
                    fr = "Reverse"
                table = [[fr, "PRIMER"],
                         ["Sequence", items.get(LoR).get("sequence")],
                     ["From... To", f"{boundaries[0]}... {boundaries[0]+boundaries[1]}"],
                     ["TM", items.get(LoR).get("TM")], 
                     ["GC", items.get(LoR).get("GC")],
                     ["Amplicon Length", items.get(LoR).get("product_size")],
                         ["Primer Size", items.get(LoR).get("size")],
                    ]
                inter_out = widgets.Output()
                with inter_out:
                    display(HTML(tabulate.tabulate(table, tablefmt='html')))

                tables.append(inter_out)
        
            griddy_2 = widgets.GridBox(children=tables,
                layout=Layout(
                width='100%',
                grid_template_rows='auto',
                grid_template_columns='50% 50%',))
            display(griddy_2)
    
    accordion = widgets.Accordion(children=[qPCR_out, PCR_out,])
    accordion.set_title(0, 'qPCR Primers')
    accordion.set_title(1, 'PCR Primers')
    display(accordion)
        
    
    
    
def create_calcPCR_butt():
    global calcPCR, all_entries
    calcPCR = widgets.Button(
    description="CALC PCR PRIMERS",
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
    )
    display(calcPCR)
    calcPCR.on_click(calcPCRout)
    
def select_transcript(widget):
    global selected_transcript, tab
    selected_transcript = (str(tab.children[tab.selected_index].children[0].value).strip("Name: "))

def printoutput(sender):
    global entered_value, a, tab, all_entries, selected_transcript
    search_query = str(a.value)
    entered_value = search_query
    clear_output()
    all_entries = dict()
    
    if search_query.startswith("SMESG"):
        output = gene_output(search_query)
        for x in list(output.rows()):
            all_entries[x["transcripts.primaryIdentifier"]] = x['transcripts.sequence.residues']
        
        tab_contents = [(f'Name: {x["transcripts.primaryIdentifier"]}', 
                         f'Sequence: {x["transcripts.sequence.residues"]}', 
                         f"Length: {len(x['transcripts.sequence.residues'])}") for x in list(output.rows())]

        children = list()
        
        for entry in tab_contents:
            items = [widgets.Label(str(entry[i])) for i in range(0,3)]
            griddy = widgets.GridBox(items)
            children.append(griddy)
        
        tab = widgets.Tab()
        tab.children = children
        for i in range(len(tab_contents)):
            tab.set_title(i, tab_contents[i][0].strip("Name: "))
                         
        display(tab)
                         
        selected_transcript = tab_contents[0][0].strip("Name: ")
        
        tab.observe(select_transcript, names='selected_index')
                             
        create_calcPCR_butt()
            
    else:
        print("NOT A VALID FORMAT - ONLY SMESGs ARE ALLOWED.")
    
   
def start():
    global entered_value, a
    a = widgets.Text(value=entered_value, disabled=False)
    display(a)
    get_list = widgets.Button(
    description="Get List",
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
    )
    display(get_list)

    get_list.on_click(printoutput)
    

In [57]:
start()                                    

Accordion(children=(Output(), Output()), _titles={'0': 'qPCR Primers', '1': 'PCR Primers'})